# Task 3: Housing Price Prediction Using Images + Tabular Data

Installing Required Libraries

In [1]:
!pip install -q torch torchvision pandas scikit-learn matplotlib pillow


Importing Libraries

In [2]:
# Core Python
import os
import numpy as np

# Data handling
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# TorchVision
from torchvision import models, transforms

# Image handling
from PIL import Image


Uploading Datasets

- Tabular data or CSV file first.

In [3]:
from google.colab import files
uploaded = files.upload()


Saving socal2.csv to socal2.csv


In [4]:
# verifying file existence

print(os.listdir())

['.config', 'socal2.csv', 'sample_data']


Uploading Images


In [5]:
from google.colab import files
uploaded_images = files.upload()


Saving 0.jpg to 0.jpg
Saving 1.jpg to 1.jpg
Saving 2.jpg to 2.jpg
Saving 3.jpg to 3.jpg
Saving 4.jpg to 4.jpg
Saving 5.jpg to 5.jpg
Saving 6.jpg to 6.jpg
Saving 7.jpg to 7.jpg
Saving 8.jpg to 8.jpg
Saving 9.jpg to 9.jpg
Saving 10.jpg to 10.jpg
Saving 11.jpg to 11.jpg
Saving 12.jpg to 12.jpg
Saving 13.jpg to 13.jpg
Saving 14.jpg to 14.jpg
Saving 15.jpg to 15.jpg
Saving 16.jpg to 16.jpg
Saving 17.jpg to 17.jpg
Saving 18.jpg to 18.jpg
Saving 19.jpg to 19.jpg
Saving 20.jpg to 20.jpg
Saving 21.jpg to 21.jpg
Saving 22.jpg to 22.jpg
Saving 23.jpg to 23.jpg
Saving 24.jpg to 24.jpg
Saving 25.jpg to 25.jpg
Saving 26.jpg to 26.jpg
Saving 27.jpg to 27.jpg
Saving 28.jpg to 28.jpg
Saving 29.jpg to 29.jpg
Saving 30.jpg to 30.jpg
Saving 31.jpg to 31.jpg
Saving 32.jpg to 32.jpg
Saving 33.jpg to 33.jpg
Saving 34.jpg to 34.jpg
Saving 35.jpg to 35.jpg
Saving 36.jpg to 36.jpg
Saving 37.jpg to 37.jpg
Saving 38.jpg to 38.jpg
Saving 39.jpg to 39.jpg
Saving 40.jpg to 40.jpg
Saving 41.jpg to 41.jpg
Saving 42.jp

In [6]:
# structuring the images in a folder

os.makedirs("images", exist_ok=True)

for filename in uploaded_images.keys():
    os.rename(filename, f"images/{filename}")


In [7]:
# verifying images

len(os.listdir("images"))

4991

Loading and Inspecting the socal2 CSV file

In [11]:
df = pd.read_csv("socal2.csv")

# matching image range
df = df[df["image_id"] < 5000].copy()

df.head()


,image_id,street,citi,n_citi,bed,bath,sqft,price
0,0,1317 Van Buren Avenue,"Salton City, CA",317,3,2.0,1560,201900
1,1,124 C Street W,"Brawley, CA",48,3,2.0,713,228500
2,2,2304 Clark Road,"Imperial, CA",152,3,1.0,800,273950
3,3,755 Brawley Avenue,"Brawley, CA",48,3,1.0,1082,350000
4,4,2207 R Carrillo Court,"Calexico, CA",55,4,3.0,2547,385100


Verifying Image-Row Matching

- this is being done as a safety check, to ensure their is an image corresponding to image_id.

In [12]:
missing = [
    f"{img}.jpg"
    for img in df["image_id"]
    if not os.path.exists(f"images/{img}.jpg")
]

print("Missing images:", missing[:10])

Missing images: ['4991.jpg', '4992.jpg', '4993.jpg', '4994.jpg', '4995.jpg', '4996.jpg', '4997.jpg', '4998.jpg', '4999.jpg']


In [13]:
# importing missing files

from google.colab import files

uploaded = files.upload()


Saving 4991.jpg to 4991.jpg
Saving 4992.jpg to 4992.jpg
Saving 4993.jpg to 4993.jpg
Saving 4994.jpg to 4994.jpg
Saving 4995.jpg to 4995.jpg
Saving 4996.jpg to 4996.jpg
Saving 4997.jpg to 4997.jpg
Saving 4998.jpg to 4998.jpg
Saving 4999.jpg to 4999.jpg


In [14]:
# moving them to the images directory

import shutil
import os

os.makedirs("images", exist_ok=True)

for fname in uploaded.keys():
    shutil.move(fname, f"images/{fname}")


In [15]:
# checking for any missing images again

missing = [
    f"{img}.jpg"
    for img in df["image_id"]
    if not os.path.exists(f"images/{img}.jpg")
]

print("Missing images:", missing[:10])

Missing images: []


No missing images, continue.

Defining Tabular Features and the Target.

In [16]:
tabular_features = [
    "bed",
    "bath",
    "sqft"
]

target = "price"


In [17]:
X_tab = df[tabular_features].values
y = df[target].values


Train-Test Split

In [18]:
X_tab_train, X_tab_test, y_train, y_test, df_train, df_test = train_test_split(
    X_tab, y, df, test_size=0.2, random_state=42
)


Scaling Tabular Data

In [19]:
scaler = StandardScaler()
X_tab_train = scaler.fit_transform(X_tab_train)
X_tab_test = scaler.transform(X_tab_test)


Creating Multimodal Dataset Class (IMP)

In [20]:
class HousingDataset(Dataset):
    def __init__(self, df, tab_data, targets, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.tab_data = tab_data
        self.targets = targets
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.loc[idx, "image_id"]
        img_path = f"{self.img_dir}/{img_id}.jpg"
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        tab = torch.tensor(self.tab_data[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)

        return image, tab, target


Image Transformations

- images are resized to 96x96 for CPU to handle training.

In [26]:
image_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor()
])


DataLoaders

In [27]:
train_dataset = HousingDataset(
    df_train, X_tab_train, y_train, "images", image_transform
)

test_dataset = HousingDataset(
    df_test, X_tab_test, y_test, "images", image_transform
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers=2)

Creating Multimodal CNN and Tabular Model

In [23]:
class MultiModalModel(nn.Module):
    def __init__(self, tab_dim):
        super().__init__()

        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

        self.tab_net = nn.Sequential(
            nn.Linear(tab_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

        self.fc = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, img, tab):
        img_feat = self.cnn(img)
        tab_feat = self.tab_net(tab)
        combined = torch.cat([img_feat, tab_feat], dim=1)
        return self.fc(combined).squeeze()


Training Models

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiModalModel(X_tab.shape[1]).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CPU Optimisation

In [29]:
torch.set_num_threads(4)  # use multiple CPU cores

In [32]:
from tqdm import tqdm

epochs = 3

patience = 3
best_loss = float('inf')
counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for imgs, tabs, targets in loop:
        imgs, tabs, targets = imgs.to(device), tabs.to(device), targets.to(device)

        optimizer.zero_grad()
        preds = model(imgs, tabs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=total_loss/len(train_loader))

    epoch_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break


Epoch 1: Loss = 652280220483.5840


Epoch 2: Loss = 651878710116.3521


Epoch 3: Loss = 650950063947.7760


Evaluation using MAE and RMSE

In [33]:
# Load best model
model = MultiModalModel(X_tab.shape[1]).to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Evaluate on test data
preds, actuals = [], []

with torch.no_grad():
    for imgs, tabs, targets in test_loader:
        imgs, tabs = imgs.to(device), tabs.to(device)
        outputs = model(imgs, tabs)
        preds.extend(outputs.cpu().numpy())
        actuals.extend(targets.numpy())

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(actuals, preds)
rmse = np.sqrt(mean_squared_error(actuals, preds))

print("MAE:", mae)
print("RMSE:", rmse)



MAE: 715559.6117929382
RMSE: 817448.706436598


# Final Summary / Insights

- Model: Combined ResNet18 for images and a fully connected network for tabular features to predict housing prices.

- Data: 5000 images and tabular features; tabular data standardized and images resized to 96×96 for faster CPU training.

- Training Setup: Adam optimizer (LR=1e-4), MSE loss, batch size 16, 3 epochs for initial testing, early stopping and best-model saving implemented.

- DataLoader Optimization: Used num_workers=2 to preload images in parallel on CPU.

- Results: Loss decreased slightly over 3 epochs; evaluation metrics—MAE ≈ 715,560, RMSE ≈ 817,449. High values due to large-scale targets and limited epochs.

- Next Steps: Scale target values, increase epochs, and tune learning rate/batch size for better convergence.